In [1]:
import pandas as pd
import scipy.stats
import numpy as np

# A/B testing
Después de tomar decisiones del tipo sí/no al elegir una estrategia, aún quedan preguntas por responder. En nuestro escenario de marketing: ¿cómo debería ser el contenido de e-mails?, ¿cómo lo diseñamos? y, ¿cómo sabemos que lo estamos haciendo bien?

Vamos a estudiar por qué es necesario crear situaciones experimentales y cómo diseñarlas.

## La necesidad de experimentación

In [2]:
desktops=pd.read_csv('d:\\cursos\\data_science\\data\\desktop.csv')
laptop=pd.read_csv('d:\\cursos\\data_science\\data\\laptop.csv')

Vamos a introducir los test para diferencias las muestras:

In [3]:
print(scipy.stats.ttest_ind(desktops['spending'],laptop['spending']))
print(scipy.stats.ttest_ind(desktops['age'],laptop['age']))
print(scipy.stats.ttest_ind(desktops['visits'],laptop['visits']))

TtestResult(statistic=-2.109853741030508, pvalue=0.03919630411621095, df=58.0)
TtestResult(statistic=-0.7101437106800108, pvalue=0.4804606394128761, df=58.0)
TtestResult(statistic=0.20626752311535543, pvalue=0.8373043059847984, df=58.0)


Aunque diferenciamos las muestras según sus gastos en desktops o laptops, es un hecho aislado y no nos dice nada acerca de cómo manejar nuestra estrategia de marketing. Pueden existir estudios que nos den pista sobre cómo avanzar, pero podría no haber información para nuestro caso particular.

Para avanzar necesitamos una nueva forma de generar datos, para lo cual recurrimos al **A/B testing**, de esta manera podrémos obtener información sobre lo que mejor funciona para nuestro negocio. **El A/B testing es un proceso que permite recolectar datos, y luego analizarlos** con métodos como el t-test. Podemos considerar los siguientes pasos:
* Diseño experimental.
* Asignación aleatoria a los grupos de caso y de control.
* Medición de los resultados.
* Comparación estadística de los resultados.

## Experimentar nuevas hipótesis
Considerémos la hipótesis sobre si un cambio de color de fuente de negro a azul incrementará las ventas. Las hipótesis relacionadas son:
* **Hipótesis 0**: cambiar el color de negro a azul no cambiará los ingresos de ventas.
* **Hipótesis 1**: cambiar el color de negro a azul cambiara los ingresos (incremento o decremento) de ventas.

En este caso, no contamos con datos para analizar y aceptar o rechazar una hipótesis, necesitamos colectar nuevos datos. Para ello, debemos diseñar un experimento, experimentar y recolectar los datos para analizar.

Primero, tenemos que considerar dos grupos: un grupo que conserva el color negro, y otro que recibe e-mails con fuente azul, y comprobar los ingresos que aporta cada individuo de cada grupo.

Vamos a separar los grupos según la media de edad:

In [4]:
median_age = desktops['age'].median()
group_a = desktops.loc[desktops['age'] <= median_age,:] # Grupo A con texto negro
group_b = desktops.loc[desktops['age'] > median_age,:] # Grupo B con texto azul

Con ambos dataframes, se pueden enviar al equipo de trabajo para mandar correos a ambos grupos y dar seguimiento de los ingresos generados por usuario (quién hace clic en los links enviados). Vamos a cargar los resultados del experimento:

In [5]:
emailresults1 = pd.read_csv('d:\\cursos\\data_science\\data\\emailresults1.csv')

Mezclamos los datasets `group_a` y `group_b` con los resultados del experimento según el id:

In [6]:
groupa_withRevenue= group_a.merge(emailresults1, on='userid')
groupb_withRevenue= group_b.merge(emailresults1, on='userid')

Realizamos el t-test para verificar si son grupos diferentes:

In [7]:
scipy.stats.ttest_ind(groupa_withRevenue['revenue'], groupb_withRevenue['revenue'])

TtestResult(statistic=-2.186454851070545, pvalue=0.03730073920038287, df=28.0)

Al considerar un niverl de $p$-value como 0.05, nos damos cuenta los grupos son diferentes. Podríamos ahora rechazar la hipótesis 0 en pro de la hipótesis 1, y diríamos que con el color de fuente cambian los ingresos, sin embargo, hemos relacionado nuestros grupos con la *edad*. Es decir, si las personas adultas tienen mejores condiciones económicas, entonces, **¡hemos estudiado las condiciones económicas según la edad!**, pero no el cambio de color.

### Buenas prácticas

Para estudiar correctamente los grupos, debemos hacer una selección aleatoria: toda muestra tiene un valor esperado igual a la media poblacional.

Vamos a hacer un estudio que considere los ingresos al incluir una imagen en los correos, pero experimentarémos con los suscriptores de laptops:

In [8]:
np.random.seed(18811015)
laptop.loc[:,'groupassignment1']=1*(np.random.random(len(laptop.index))>0.5) # Columna aleatoria de 1s y 0s
groupc = laptop.loc[laptop['groupassignment1'] == 0,:].copy() # Asignar grupo con 0s
groupd = laptop.loc[laptop['groupassignment1'] == 1,:].copy() # Asignar grupo con 1s

Con el dataframe de los grupos aleatorios, enviamos al equipo de trabajo y experimentar. Vamos a cargar los resultados:

In [9]:
emailresults2 = pd.read_csv('d:\\cursos\\data_science\\data\\emailresults2.csv')

In [10]:
groupc_withrevenue = groupc.merge(emailresults2, on='userid' )
groupd_withrevenue = groupd.merge(emailresults2, on='userid' )

Hagamos el t-test para obtener el $p$-value, considerando un nivel significativo de 0.5:

In [11]:
scipy.stats.ttest_ind(groupc_withrevenue['revenue'], groupd_withrevenue['revenue'])

TtestResult(statistic=-2.381320497676198, pvalue=0.024288828555138562, df=28.0)

Nuestra diferencia es menor que 0.5, por tanto, podemos aceptar que se trata de grupos diferentes y que hay un efecto en el cambio de ingresos. Si nuestro equipo de trabajo envió imágenes al grupo D, podemos calcular el efecto:

In [12]:
np.mean(groupd_withrevenue['revenue'])-np.mean(groupc_withrevenue['revenue'])

260.3333333333333

## Effect sizes
El valor calculado de $\$260$ de diferencia entre las medias en nuestro último test se conoce como effect size. Una forma de determinar si el effect size es pequeño, medio o grande es mediante el número $d$ de Cohen:

$$\frac{\text{effect size}}{\sigma}$$

Que representa el *número de desviaciones estándar* en que difieren las medias de dos poblaciones. Agunas convencions de valor para la $d$ de Cohen:
* Valor bajo: $d\leq2$.
* Valor medio: $d\approx0.5$.
* Valor alto: $0.8\leq d$.

Por ejemplo, la $d$ de Cohen para nuestro primer test sugiere que es un valor alto:

In [13]:
125/np.std(emailresults1['revenue'])

0.763769235188029